In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import skfuzzy as fuzz
import skfuzzy.control as ctrl
import random
from collections import deque

# Load cleaned dataset
data = pd.read_csv("cleaned_data.csv")

data_sampled, _ = train_test_split(data, test_size=0.998, stratify=data['anomaly'], random_state=42)
print(f"Reduced dataset size: {data_sampled.shape[0]} rows")

X = data_sampled[['meter_reading']].values
y = data_sampled['anomaly'].values

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

meter_reading = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'meter_reading')
anomaly = ctrl.Antecedent(np.arange(0, 2, 1), 'anomaly')
reward = ctrl.Consequent(np.arange(-1, 2, 0.1), 'reward')

meter_reading['low'] = fuzz.trapmf(meter_reading.universe, [0, 0, 0.25, 0.5])
meter_reading['medium'] = fuzz.trimf(meter_reading.universe, [0.25, 0.5, 0.75])
meter_reading['high'] = fuzz.trapmf(meter_reading.universe, [0.5, 0.75, 1.0, 1.0])

# Membership functions for anomaly
anomaly['normal'] = fuzz.trimf(anomaly.universe, [0, 0, 1])
anomaly['abnormal'] = fuzz.trimf(anomaly.universe, [0, 1, 1])

reward['negative'] = fuzz.trapmf(reward.universe, [-1, -1, -0.5, 0])
reward['neutral'] = fuzz.trimf(reward.universe, [-0.5, 0, 0.5])
reward['positive'] = fuzz.trapmf(reward.universe, [0, 0.5, 1, 1])

rule1 = ctrl.Rule(meter_reading['low'] & anomaly['normal'], reward['positive'])
rule2 = ctrl.Rule(meter_reading['medium'] & anomaly['abnormal'], reward['negative'])
rule3 = ctrl.Rule(meter_reading['high'], reward['neutral'])

reward_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
reward_system = ctrl.ControlSystemSimulation(reward_ctrl)

class AnomalyDetectionEnv:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.current_index = 0
        self.state = self.X[self.current_index]

    def reset(self):
        self.current_index = 0
        indices = np.arange(len(self.X))
        np.random.shuffle(indices)
        self.X, self.y = self.X[indices], self.y[indices]
        self.state = self.X[self.current_index]
        return self.state

    def step(self, action):
        """
        Take an action and return the next state, reward, and whether the episode is done.

        Args:
            action (int): The predicted label (0: normal, 1: abnormal).

        Returns:
            next_state (np.array): The next state.
            reward (int): Reward (1 for correct classification, 0 for incorrect classification).
            done (bool): Whether the episode has ended.
        """
        # Calculate the reward
        correct_label = self.y[self.current_index]
        reward = 1 if action == correct_label else 0

        self.current_index += 1
        done = self.current_index >= len(self.X) or self.current_index >= 100  # Limit episode length

        if not done:
            self.state = self.X[self.current_index]
        else:
            self.state = None  

        return self.state, reward, done

class DRLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.main_model = self._build_model()  # Main network
        self.target_model = self._build_model()  # Target network
        self.update_target_network()
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.update_target_freq = 5  # Frequency to update target network

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.Input(shape=(self.state_size,)),
            layers.Dense(24, activation='relu'),
            layers.Dense(24, activation='relu'),
            layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
        return model

    def update_target_network(self):
        self.target_model.set_weights(self.main_model.get_weights())

    def fuzzy_predict_action(self, state):
        """
        Predict action using the fuzzy logic system based on the current state.
        """
        try:
            reward_system.input['meter_reading'] = state[0]
            reward_system.input['anomaly'] = 0  
            reward_system.compute()
            fuzzy_reward = reward_system.output['reward']

            return 1 if fuzzy_reward > 0 else 0
        except KeyError as e:
            print(f"KeyError in fuzzy system: {e}")
            return 0  # Default action in case of error

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.action_size)
        q_values = self.main_model.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                # Use fuzzy logic to predict the next action
                predicted_action = self.fuzzy_predict_action(next_state[0])
                q_next = self.target_model.predict(next_state, verbose=0)[0][predicted_action]
                target += self.gamma * q_next
            target_f = self.main_model.predict(state, verbose=0)
            target_f[0][action] = target
            self.main_model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def train(self, episodes, batch_size, env):
        for e in range(episodes):
            state = env.reset()
            state = np.reshape(state, [1, self.state_size])

            for time in range(100):  # Limit steps per episode
                action = self.act(state)
                next_state, reward, done = env.step(action)
                next_state = np.reshape(next_state, [1, self.state_size])
                self.remember(state, action, reward, next_state, done)
                state = next_state
                if done:
                    print(f"Episode {e+1}/{episodes}, Steps {time}")
                    break
            self.replay(batch_size)

            # Update the target network periodically using fuzzy predictions
            if e % self.update_target_freq == 0:
                self.update_target_network()

env = AnomalyDetectionEnv(X_train, y_train)
agent = DRLAgent(state_size=X_train.shape[1], action_size=2)

episodes = 100
batch_size = 32
agent.train(episodes, batch_size, env)

# Evaluate the model
y_pred = []
for i in range(len(X_test)):
    state = np.reshape(X_test[i], [1, agent.state_size])
    action = agent.act(state)
    y_pred.append(action)

print("Classification Report:")
print(classification_report(y_test, y_pred))

# Compute accuracy manually using the confusion matrix
def calculate_accuracy(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    acc = (tp + tn) / (tp + tn + fp + fn)
    return acc, tp, tn, fp, fn

accuracy, tp, tn, fp, fn = calculate_accuracy(y_test, y_pred)

print("\nConfusion Matrix Results:")
print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")

print(f"\nAccuracy (ACC): {accuracy:.2f}")



Reduced dataset size: 3250 rows
Episode 1/100, Steps 99
Episode 2/100, Steps 99
Episode 3/100, Steps 99
Episode 4/100, Steps 99
Episode 5/100, Steps 99
Episode 6/100, Steps 99
Episode 7/100, Steps 99
Episode 8/100, Steps 99
Episode 9/100, Steps 99
Episode 10/100, Steps 99
Episode 11/100, Steps 99
Episode 12/100, Steps 99
Episode 13/100, Steps 99
Episode 14/100, Steps 99
Episode 15/100, Steps 99
Episode 16/100, Steps 99
Episode 17/100, Steps 99
Episode 18/100, Steps 99
Episode 19/100, Steps 99
Episode 20/100, Steps 99
Episode 21/100, Steps 99
Episode 22/100, Steps 99
Episode 23/100, Steps 99
Episode 24/100, Steps 99
Episode 25/100, Steps 99
Episode 26/100, Steps 99
Episode 27/100, Steps 99
Episode 28/100, Steps 99
Episode 29/100, Steps 99
Episode 30/100, Steps 99
Episode 31/100, Steps 99
Episode 32/100, Steps 99
Episode 33/100, Steps 99
Episode 34/100, Steps 99
Episode 35/100, Steps 99
Episode 36/100, Steps 99
Episode 37/100, Steps 99
Episode 38/100, Steps 99
Episode 39/100, Steps 99
Ep

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Train an SVM model
svm_model = SVC(kernel='rbf', gamma='scale', random_state=42)
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_svm))

accuracy, tp, tn, fp, fn = calculate_accuracy(y_test, y_pred_svm)


print("\nConfusion Matrix Results:")
print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")

print(f"\nAccuracy (ACC): {accuracy:.2f}")

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       637
           1       0.00      0.00      0.00        13

    accuracy                           0.98       650
   macro avg       0.49      0.50      0.49       650
weighted avg       0.96      0.98      0.97       650


Confusion Matrix Results:
True Positives (TP): 0
True Negatives (TN): 637
False Positives (FP): 0
False Negatives (FN): 13

Accuracy (ACC): 0.98


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

X_train_cnn = X_train.reshape(-1, X_train.shape[1], 1)
X_test_cnn = X_test.reshape(-1, X_test.shape[1], 1)

# Build the CNN model
cnn_model = models.Sequential([
    layers.Conv1D(32, 1, activation='relu', input_shape=(X_train.shape[1], 1)),  # Kernel size = 1
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

cnn_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_data=(X_test_cnn, y_test))

y_pred_cnn = (cnn_model.predict(X_test_cnn) > 0.5).astype("int32")

print("CNN Classification Report:")
print(classification_report(y_test, y_pred_cnn))

accuracy, tp, tn, fp, fn = calculate_accuracy(y_test, y_pred_cnn)

print("\nConfusion Matrix Results:")
print(f"True Positives (TP): {tp}")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")

print(f"\nAccuracy (ACC): {accuracy:.2f}")




Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


407/407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9625 - loss: 0.3419 - val_accuracy: 0.9803 - val_loss: 0.0981
Epoch 2/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9779 - loss: 0.1041 - val_accuracy: 0.9803 - val_loss: 0.0975
Epoch 3/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9779 - loss: 0.1042 - val_accuracy: 0.9803 - val_loss: 0.0986
Epoch 4/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9790 - loss: 0.1014 - val_accuracy: 0.9803 - val_loss: 0.0980
Epoch 5/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9749 - loss: 0.1162 - val_accuracy: 0.9803 - val_loss: 0.0976
Epoch 6/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9756 - loss: 0.1145 - val_accuracy: 0.9803 - val_loss: 0.0982
Epoch 7/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9773 - loss: 0.1051 - val_accuracy: 0.9803 - val_loss: 0.0970
Epoch 8/10
407/407 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9764 - loss: 0.1100 - val_accuracy: 0.9803 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
